In [1]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf

# Load the TFLite model
model_path = "manipulation_detector_config_1.tflite"
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Preprocesses an image before it is given to the model to ensure consistency
# across all images that the model is learning off of.
def preprocess_image(image_path):
    # Load the image and convert it to RGB colour space; 
    # we want all images to be in same colour space for consistency.
    with Image.open(image_path) as image:
        image = image.convert("RGB")
    # Resize the image to a standard size of 256 x 256
    image = image.resize((256, 256))
    # Convert the image to an array. Normalize its pixel values to be between 0 and 1.
    # Since most images in the training/test dataset are jpegs and in RGB, assume that
    # we can divide each pixel by 255 to normalize them to be between 0 and 1.
    preprocessed_image = np.array(image) / 255.0
    # Return the preprocessed image.
    return preprocessed_image

# Define a function to classify a single image
def classify_image(image_path):
    input_data = preprocess_image(image_path)
    input_data = np.expand_dims(input_data, axis=0).astype(np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data[0][0]

# Define the folder where the images are located
folder_path = "reserve"

# Initialize counters for classification results
num_authentic = 0
num_manipulated = 0
num_au_authentic = 0
num_au_manipulated = 0
num_tp_authentic = 0
num_tp_manipulated = 0
num_au_filenames = 0
num_tp_filenames = 0

# Iterate over all the images in the folder
for filename in os.listdir(folder_path):
    if (filename.endswith(".jpg") or filename.endswith(".jpeg")):
        # Classify the image and update counters
        image_path = os.path.join(folder_path, filename)
        classification_result = classify_image(image_path)
        if classification_result < 0.5:
            print(f"{filename} - Manipulated: {classification_result}")
            num_manipulated += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_manipulated += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_manipulated += 1
                num_tp_filenames += 1
        else:
            print(f"{filename} - Authentic: {classification_result}")
            num_authentic += 1
            if (filename.startswith("Au") or filename.startswith("au")):
                num_au_authentic += 1
                num_au_filenames += 1
            elif (filename.startswith("Tp") or filename.startswith("tp")):
                num_tp_authentic += 1
                num_tp_filenames += 1

# Print the classification results and filename counts
print(f"Total authentic images: {num_authentic}")
print(f"Total manipulated images: {num_manipulated}")
print(f"Number of 'au' authentic images: {num_au_authentic}")
print(f"Number of 'au' manipulated images: {num_au_manipulated}")
print(f"Number of 'tp' authentic images: {num_tp_authentic}")
print(f"Number of 'tp' manipulated images: {num_tp_manipulated}")
print(f"Number of 'au' filenames: {num_au_filenames}")
print(f"Number of 'tp' filenames: {num_tp_filenames}")

precision = num_au_authentic/(num_au_authentic + num_tp_authentic)
recall = num_au_authentic/(num_au_authentic + num_au_manipulated)
accuracy = (num_au_authentic + num_tp_manipulated)/(num_au_authentic + num_tp_authentic + num_au_manipulated + num_tp_manipulated)

print(f"precision: {precision}")
print(f"recall: {recall}")
print(f"accuracy: {accuracy}")



Au_pla_30535.jpg - Authentic: 1.0
Tp_S_NRD_S_N_pla20018_pla20018_02489.jpeg - Manipulated: 0.22794662415981293
Au_pla_30253.jpg - Authentic: 0.9998343586921692
Au_pla_20040.jpg - Manipulated: 0.12850317358970642
Au_pla_30247.jpg - Authentic: 1.0
Au_pla_20054.jpg - Manipulated: 0.034602101892232895
Au_pla_30521.jpg - Authentic: 1.0
Tp_S_NRN_S_N_pla00065_pla00065_01172.jpeg - Authentic: 0.9999924302101135
Au_pla_30509.jpg - Authentic: 0.9999357461929321
Au_arc_30592.jpg - Authentic: 0.9999996423721313
Tp_S_NRN_S_N_ind00055_ind00055_00905.jpeg - Authentic: 1.0
Tp_D_CRN_S_N_sec00071_art00028_11281.jpg - Manipulated: 0.009542744606733322
Tp_D_NNN_L_N_ani00010_nat00095_10218.jpeg - Authentic: 0.998624861240387
Au_pla_20068.jpg - Manipulated: 0.13762466609477997
Au_arc_30586.jpg - Authentic: 0.9999927282333374
Au_nat_30570.jpg - Authentic: 0.9999999403953552
Au_nat_30216.jpg - Authentic: 0.9998868107795715
Au_nat_20005.jpg - Manipulated: 0.17001992464065552
Au_pla_30290.jpg - Authentic: 0.997

Au_sec_00042.jpg - Authentic: 0.996487557888031
Au_pla_30468.jpg - Authentic: 0.9997447729110718
Au_sec_30195.jpg - Authentic: 0.9999999403953552
Au_sec_00056.jpg - Authentic: 0.9999881386756897
Au_pla_30332.jpg - Authentic: 0.998917281627655
Tp_D_NNN_M_B_nat00004_nat00095_11488.jpg - Manipulated: 0.03910991549491882
Tp_D_CRN_M_N_ani10112_ani00100_11648.jpg - Manipulated: 0.0003303989360574633
Au_pla_30454.jpg - Authentic: 0.9999906420707703
Au_cha_00062.jpg - Authentic: 0.9999940395355225
Au_pla_30440.jpg - Authentic: 0.9999998211860657
Au_pla_30326.jpg - Authentic: 0.9999991059303284
Au_sec_30426.jpg - Authentic: 0.9995021820068359
Au_arc_30632.jpg - Authentic: 0.9999997615814209
Au_nat_30149.jpg - Authentic: 0.9998753070831299
Au_sec_30340.jpg - Authentic: 0.9999931454658508
Tp_S_NRN_S_N_art20066_art20066_01872.jpeg - Manipulated: 0.09939707070589066
Au_sec_30354.jpg - Authentic: 0.9815347194671631
Au_pla_00018.jpg - Authentic: 0.9998904466629028
Tp_S_NRN_S_N_nat00077_nat00077_00983

Au_pla_30052.jpg - Authentic: 0.9996467232704163
Au_sec_30593.jpg - Authentic: 0.9999999403953552
Au_arc_30787.jpg - Authentic: 1.0
Au_arc_30793.jpg - Authentic: 0.999938428401947
Tp_D_CRN_S_N_nat00099_ani00017_10220.jpeg - Authentic: 0.981439471244812
Au_sec_30587.jpg - Authentic: 0.9996768832206726
Au_pla_30708.jpg - Authentic: 0.9979973435401917
Tp_S_NRN_S_N_arc20083_arc20083_01723.jpeg - Manipulated: 0.027860131114721298
Tp_D_CRN_S_N_cha00075_cha00086_10868.jpg - Manipulated: 0.0016674306243658066
Au_nat_30003.jpg - Authentic: 0.9999992847442627
Au_cha_30570.jpg - Authentic: 0.9999811053276062
Au_sec_20019.jpg - Manipulated: 0.1408253014087677
Tp_S_NRN_S_N_arc20080_arc20080_01720.jpeg - Manipulated: 0.0890735611319542
Au_pla_30085.jpg - Authentic: 0.9999944567680359
Au_sec_30578.jpg - Authentic: 0.999994695186615
Au_cha_30564.jpg - Authentic: 1.0
Au_pla_30091.jpg - Manipulated: 0.14313672482967377
Au_nat_30017.jpg - Authentic: 0.9999997019767761
Au_sec_20025.jpg - Manipulated: 0.15

Tp_D_CRN_M_N_nat00084_nat00099_10071.jpeg - Authentic: 0.9988886117935181
Au_nat_30638.jpg - Authentic: 0.9998963475227356
Au_sec_30431.jpg - Authentic: 0.9999191761016846
Au_sec_30357.jpg - Authentic: 0.9999995231628418
Tp_S_NRN_S_N_arc00009_arc00009_00229.jpeg - Authentic: 1.0
Au_nat_00089.jpg - Authentic: 0.9999217391014099
Tp_D_CNN_S_N_cha10122_nat10124_12167.jpg - Manipulated: 0.20895691215991974
Au_sec_30343.jpg - Authentic: 1.0
Au_sec_30425.jpg - Authentic: 1.0
Au_arc_30631.jpg - Authentic: 0.9999976754188538
Au_pla_30682.jpg - Authentic: 0.985148012638092
Au_arc_30619.jpg - Authentic: 0.9999978542327881
Au_nat_30604.jpg - Authentic: 0.999919056892395
Au_nat_30162.jpg - Authentic: 1.0
Au_pla_00027.jpg - Authentic: 0.9076445698738098
Tp_S_NRN_S_N_pla00016_pla00016_10961.jpg - Manipulated: 1.583103949087672e-05
Au_pla_00033.jpg - Authentic: 0.9999765157699585
Au_nat_30176.jpg - Authentic: 0.9994315505027771
Au_nat_30610.jpg - Authentic: 0.999970555305481
Au_pla_30696.jpg - Authent

Au_arc_30782.jpg - Authentic: 0.99996018409729
Au_arc_30796.jpg - Authentic: 0.9999587535858154
Au_sec_30582.jpg - Authentic: 0.9999983906745911
Au_pla_30725.jpg - Authentic: 0.999474823474884
Au_pla_30043.jpg - Authentic: 0.9989379644393921
Au_pla_30057.jpg - Authentic: 0.9990946054458618
Au_sec_10108.jpg - Manipulated: 0.2695133686065674
Au_pla_30731.jpg - Authentic: 0.9996386170387268
Au_sec_30555.jpg - Authentic: 0.9999999403953552
Au_sec_30233.jpg - Authentic: 0.9795877933502197
Tp_D_CNN_S_N_cha00044_cha00043_00364.jpeg - Authentic: 1.0
Au_sec_20020.jpg - Manipulated: 0.3595743775367737
Au_cha_30549.jpg - Authentic: 1.0
Tp_D_CNN_S_N_nat00094_nat00067_10612.jpg - Manipulated: 0.007795901037752628
Au_sec_30227.jpg - Authentic: 0.9999959468841553
Au_sec_20034.jpg - Manipulated: 0.3126492202281952
Tp_S_NRN_S_N_nat00018_nat00018_10927.jpg - Authentic: 0.6575757265090942
Au_sec_30541.jpg - Authentic: 0.9999973773956299
Au_sec_30569.jpg - Authentic: 1.0
Au_nat_30006.jpg - Authentic: 0.99

...

Au_sec_30175.jpg - Authentic: 1.0
Au_sec_30613.jpg - Authentic: 0.9999997019767761
Tp_S_NRN_S_N_sec00059_sec00059_11268.jpg - Manipulated: 0.43389660120010376
Tp_S_NRD_S_B_cha00089_ani00013_00409.jpeg - Authentic: 0.8189408779144287
Au_sec_30174.jpg - Authentic: 0.9999998807907104
Tp_D_CRN_L_N_art00028_art00028_11282.jpg - Manipulated: 0.2590506672859192
Au_sec_30612.jpg - Authentic: 1.0
Tp_S_NRD_S_N_ind00028_ind00028_00895.jpeg - Authentic: 0.9999986886978149
Au_sec_30606.jpg - Authentic: 0.9999841451644897
Au_pla_30489.jpg - Authentic: 0.9989486932754517
Tp_D_CRN_S_N_ani00008_ani00011_10221.jpeg - Authentic: 0.9998637437820435
Au_nat_30369.jpg - Authentic: 0.9999836087226868
Au_sec_30160.jpg - Authentic: 0.9999998211860657
Tp_D_NND_M_B_sec00077_txt00059_10387.jpeg - Authentic: 0.9905478358268738
Au_nat_30341.jpg - Authentic: 0.9960671067237854
Tp_S_NRN_S_B_ani10001_ani10001_20004.jpg - Manipulated: 0.10844384133815765
Au_sec_30148.jpg - Authentic: 1.0
Au_nat_30427.jpg - Authentic: 0.

Au_pla_30512.jpg - Authentic: 1.0
Tp_S_NRN_S_N_art20025_art20025_01831.jpeg - Manipulated: 0.3369678854942322
Au_arc_30589.jpg - Authentic: 0.9995858669281006
Au_nat_30594.jpg - Authentic: 0.9999309778213501
Tp_D_CRN_M_N_ani10120_sec00098_11632.jpg - Manipulated: 0.00019275206432212144
Tp_D_CRN_S_N_ani10116_sec00098_11616.jpg - Manipulated: 0.12844298779964447
Au_pla_20067.jpg - Manipulated: 0.11570417881011963
Au_pla_30274.jpg - Authentic: 0.9532190561294556
Au_sec_30010.jpg - Authentic: 0.9979760050773621
Au_nat_30219.jpg - Authentic: 0.9934032559394836
Tp_D_NNN_M_N_arc00008_nat00059_11772.jpg - Manipulated: 0.2824647128582001
Au_arc_30562.jpg - Authentic: 1.0
Au_cha_30018.jpg - Authentic: 0.9992374777793884
Au_arc_30576.jpg - Authentic: 0.9999998807907104
Au_pla_20098.jpg - Authentic: 0.8972809910774231
Tp_S_NRN_S_N_arc20085_arc20085_02416.jpeg - Manipulated: 0.3934301435947418
Au_sec_30004.jpg - Authentic: 1.0
Au_nat_20036.jpg - Manipulated: 0.3146459758281708
Au_nat_30225.jpg - Au

Tp_S_NRN_S_N_ind20048_ind20048_01789.jpeg - Authentic: 0.5789059400558472
Au_sec_00071.jpg - Authentic: 1.0
Au_sec_00065.jpg - Authentic: 0.9999979734420776
Au_pla_30329.jpg - Authentic: 0.9999992847442627
Au_cha_00079.jpg - Authentic: 0.9958084225654602
Tp_D_CRN_S_O_cha00029_ani00068_10041.jpeg - Authentic: 1.0
Au_pla_00003.jpg - Authentic: 0.9992568492889404
Au_nat_30146.jpg - Authentic: 1.0
Au_nat_00085.jpg - Authentic: 0.9416508674621582
Au_nat_30620.jpg - Authentic: 0.93710857629776
Au_sec_30429.jpg - Authentic: 0.9996514320373535
Au_arc_30629.jpg - Authentic: 0.9999943971633911
Au_nat_30634.jpg - Authentic: 0.9999963641166687
Au_nat_30152.jpg - Authentic: 0.9999275803565979
Au_nat_00091.jpg - Authentic: 0.6628236174583435
Au_pla_00017.jpg - Authentic: 0.9547953605651855
Tp_S_NRN_S_N_art00095_art00095_01386.jpeg - Authentic: 0.9999502301216125
Au_sec_30373.jpg - Authentic: 0.9996110796928406
Au_sec_30415.jpg - Authentic: 0.999973475933075
Au_arc_30601.jpg - Authentic: 1.0
Tp_D_NNN

Au_sec_20002.jpg - Manipulated: 0.3146122395992279
Au_sec_30577.jpg - Authentic: 0.9999998807907104
Tp_S_NRN_S_N_arc20001_arc20001_02464.jpeg - Manipulated: 0.15179219841957092
Tp_S_NRN_S_B_cha10211_cha10211_12330.jpg - Manipulated: 0.23786579072475433
Tp_D_CRN_M_N_txt00050_txt00049_10368.jpeg - Authentic: 0.9999045133590698
Au_nat_30030.jpg - Authentic: 0.9993672966957092
Au_sec_30239.jpg - Authentic: 0.9999039769172668
Au_cha_30557.jpg - Authentic: 1.0
Tp_S_NRN_S_N_sec20020_sec20020_01610.jpeg - Authentic: 0.6879909038543701
Au_nat_30024.jpg - Authentic: 0.9991986155509949
Tp_S_NRN_S_N_nat20033_nat20033_02233.jpeg - Manipulated: 0.44035759568214417
Au_txt_00026.jpg - Authentic: 0.999999463558197
Au_nat_00078.jpg - Authentic: 0.9998503923416138
Au_pla_30129.jpg - Authentic: 1.0
Tp_S_NRD_S_N_sec20063_sec20063_01653.jpeg - Authentic: 0.743389904499054
Au_txt_00032.jpg - Authentic: 0.999991238117218
Au_pla_30667.jpg - Authentic: 0.999843180179596
Au_pla_30101.jpg - Authentic: 0.999998331

Au_nat_00079.jpg - Authentic: 0.9625491499900818
Au_nat_30025.jpg - Authentic: 0.9999744892120361
Au_cha_30556.jpg - Authentic: 1.0
Tp_S_NRN_S_N_ind20030_ind20030_01771.jpeg - Manipulated: 0.43123283982276917
Au_sec_30238.jpg - Authentic: 0.9995369911193848
Au_nat_30031.jpg - Authentic: 0.9984274506568909
Au_sec_20003.jpg - Manipulated: 0.3664402365684509
Au_sec_30210.jpg - Authentic: 0.9999998807907104
Tp_S_NRN_S_N_nat10147_nat10147_11995.jpg - Manipulated: 0.07821617275476456
Au_nat_30019.jpg - Authentic: 0.9763945937156677
Au_sec_30576.jpg - Authentic: 1.0
Tp_S_NRN_S_N_nat00082_nat00082_00986.jpeg - Authentic: 0.99507737159729
Au_sec_30562.jpg - Authentic: 1.0
Tp_S_NRN_S_B_cha00086_art00012_00406.jpeg - Authentic: 0.9990456700325012
Au_sec_20017.jpg - Manipulated: 0.12656493484973907
Au_sec_30204.jpg - Authentic: 1.0
Au_pla_30060.jpg - Authentic: 0.9999891519546509
Au_pla_30706.jpg - Authentic: 0.9999724626541138
Au_sec_30589.jpg - Authentic: 1.0
Au_pla_30712.jpg - Authentic: 0.9997